In [2]:
import os
import json
os.environ["PYSPARK_PIN_THREAD"] = "true"
from typing import List, Optional

from kafka import KafkaConsumer
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from delta.tables import DeltaTable

# Définir le schéma pour le JSON
OFFRE_SCHEMA = StructType([
    StructField("titre_du_poste", StringType(), True),
    StructField("societe", StringType(), True),
    StructField("competences", ArrayType(StringType()), True),
    StructField("lieu", StringType(), True),
    StructField("type_offre", StringType(), True),
    StructField("durée", StringType(), True),
    StructField("type_de_contrat", StringType(), True),
    StructField("email", StringType(), True),
    StructField("telephone", StringType(), True),
    StructField("type", StringType(), True),
    StructField("langues", ArrayType(StringType()), True),
    StructField("salaire", StringType(), True),
    StructField("date_de_debut", StringType(), True),
    StructField("secteur_dactivite", StringType(), True),
    StructField("experience_demande", StringType(), True),
    StructField("formation_requise", StringType(), True),
    StructField("avantages", ArrayType(StringType()), True),
    StructField("site_web", StringType(), True)
])

# Variables d'environnement (à remplacer par des valeurs sécurisées en production)
GROQ_API_KEY = "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"
ADLS_STORAGE_ACCOUNT_NAME = "dataoffre"
ADLS_ACCOUNT_KEY = "1eNXm2As1DuaMeSt2Yjegn22fFCvIUa8nBhknayEyTgfBZb6xEEyZhnvl9OiGT7U4O7cFrygjBE/+ASt1hkNQQ=="  # A remplacer par votre clé
ADLS_CONTAINER_NAME = "postes"
ADLS_FOLDER_PATH = "offres_trav"

KAFKA_BOOTSTRAP_SERVERS = "pkc-619z3.us-east1.gcp.confluent.cloud:9092"
KAFKA_GROUP_ID = "groupe_traitement"
KAFKA_API_KEY = "OM3FCB4RLKF3L2AQ"
KAFKA_API_SECRET = "TaIh3NYZANuLKfatv3dHcQLFaigVQvIdG+uY9Sma/eFIPzMXCWvdojhc6Q1+/BWK"
KAFKA_TOPIC = "offres_trav"

OUTPUT_PATH = f"abfss://{ADLS_CONTAINER_NAME}@{ADLS_STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/{ADLS_FOLDER_PATH}"

# Packages requis pour Spark
PACKAGES = [
    "io.delta:delta-spark_2.12:1.1.0",
    "org.apache.hadoop:hadoop-azure:3.3.6",
    "org.apache.hadoop:hadoop-azure-datalake:3.3.6",
]

def create_or_get_spark(app_name: str, packages: List[str]) -> SparkSession:
    """Créer une session Spark avec Delta et Azure."""
    jars = ",".join(packages)
    spark = (
        SparkSession.builder.appName(app_name)
        .config("spark.jars.packages", jars)
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("fs.abfss.impl", "org.apache.hadoop.fs.azurebfs.SecureAzureBlobFileSystem")
        .master("local[*]")
        .getOrCreate()
    )
    spark.conf.set(
        f"fs.azure.account.key.{ADLS_STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
        ADLS_ACCOUNT_KEY,
    )
    return spark

def create_empty_delta_table(
    spark: SparkSession,
    schema: StructType,
    path: str,
    partition_cols: Optional[List[str]] = None,
    enable_cdc: Optional[bool] = False,
):
    """Créer une table Delta vide si elle n'existe pas."""
    try:
        DeltaTable.forPath(spark, path)
        print(f"Delta Table already exists at path: {path}")
    except Exception:
        print(f"Creating new Delta Table at: {path}")
        custom_builder = DeltaTable.createIfNotExists(spark).location(path).addColumns(schema)
        if partition_cols:
            custom_builder = custom_builder.partitionedBy(partition_cols)
        if enable_cdc:
            custom_builder = custom_builder.property("delta.enableChangeDataFeed", "true")
        custom_builder.execute()

def save_to_delta(df: DataFrame, output_path: str):
    """Sauvegarder les données dans une Delta Table."""
    df.write.format("delta").mode("append").option("mergeSchema", "true").save(output_path)
    print("Data written to Delta Table.")

def process_message(spark: SparkSession, message: str):
    """Traiter un message Kafka et l'enregistrer dans une table Delta."""
    try:
        job_posting = json.loads(message)
        df = spark.createDataFrame([job_posting], schema=OFFRE_SCHEMA)
        save_to_delta(df, OUTPUT_PATH)
    except Exception as e:
        print(f"Error processing message: {e}")

# Configurer le consommateur Kafka pour Confluent
consumer = KafkaConsumer(
    KAFKA_TOPIC,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    group_id=KAFKA_GROUP_ID,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    security_protocol="SASL_SSL",
    sasl_mechanism="PLAIN",
    sasl_plain_username=KAFKA_API_KEY,
    sasl_plain_password=KAFKA_API_SECRET,
)

# Entrée principale
if __name__ == "__main__":
    # Créer une session Spark
    spark = create_or_get_spark("json_to_delta", PACKAGES)
    create_empty_delta_table(spark, OFFRE_SCHEMA, OUTPUT_PATH, partition_cols=["societe"])

    print("Listening to Kafka topic...")
    for message in consumer:
        message_value = message.value.decode("utf-8")
        process_message(spark, message_value)


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [1]:
!pip install pyspark

In [ ]:



from pyspark.sql import SparkSession

# Créer une session Spark avec plus de ressources
spark = SparkSession.builder \
    .appName("fadwa") \
    .master("spark://localhost:7077").config("spark.executor.memory", "2g").config("spark.driver.memory", "2g") .config("spark.executor.cores", "2")  .config("spark.cores.max", "4")      .config("spark.sql.shuffle.partitions", "4").getOrCreate()
print(spark)
# Créer un DataFrame simple
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["Name", "Value"])

# Afficher le DataFrame
df.show()

# Effectuer une transformation simple
df_filtered = df.filter(df["Value"] > 1)

# Afficher le DataFrame filtré
df_filtered.show()

# Stopper la session Spark
spark.stop()


In [7]:
from pyspark.sql import SparkSession

# Vérifier et redémarrer Spark si nécessaire
if 'spark' in locals() and spark.sparkContext._jsc.sc().isStopped():
    spark.stop()

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MyApp") \
    .getOrCreate()

# Créer un DataFrame simple
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["Name", "Value"])

# Afficher le DataFrame
df.show()


Py4JJavaError: An error occurred while calling o136.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (DESKTOP-64EAN51 executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=2, Le fichier spécifié est introuvable
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartition

In [2]:
from pyspark import SparkContext, SparkConf
import os
os.environ["PYSPARK_PYTHON"] = "python"

# Créer une configuration Spark
conf = SparkConf().setAppName("WordCount").setMaster("local[*]")
sc = SparkContext(conf=conf)

# Créer un RDD avec une liste de mots
data = sc.parallelize(["Spark", "Scala", "Hadoop", "Big Data", "Spark"])

# Compter le nombre d'occurrences de chaque mot
word_counts = data.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)

# Afficher les résultats
for word, count in word_counts.collect():
    print(f"{word}: {count}")

# Arrêter le contexte Spark
sc.stop()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=WordCount, master=local[*]) created by __init__ at C:\Users\HP\AppData\Local\Temp\ipykernel_2632\1807962369.py:5 

In [1]:
import os
from typing import List, Optional
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_PIN_THREAD"] = "true"



# Définir le schéma pour le JSON
PERSON_SCHEMA = StructType([
   
    StructField("last_name", StringType(), True),
    StructField("full_name", StringType(), True),
    StructField("title", StringType(), True),
    StructField("address", StructType([
        StructField("formatted_location", StringType(), True),
        StructField("city", StringType(), True),
        StructField("region", StringType(), True),
        StructField("country", StringType(), True),
        StructField("postal_code", StringType(), True)
    ]), True),
    StructField("objective", StringType(), True),
    StructField("date_of_birth", StringType(), True),
    StructField("place_of_birth", StringType(), True),
    StructField("phones", StringType(), True),
    StructField("urls", StructType([
        StructField("GitHub", StringType(), True),
        StructField("portfolio", StringType(), True),
        StructField("LinkedIn", StringType(), True),
        StructField("site_web", StringType(), True)
    ]), True),
    StructField("gender", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("education_details", ArrayType(StructType([
        StructField("etude_title", StringType(), True),
        StructField("etablissement_name", StringType(), True),
        StructField("start_date", StringType(), True),
        StructField("end_date", StringType(), True),
        StructField("etude_city", StringType(), True),
        StructField("etude_region", StringType(), True),
        StructField("end_country", StringType(), True)
    ])), True),
    StructField("work_experience_details", ArrayType(StructType([
        StructField("job_title", StringType(), True),
        StructField("company_name", StringType(), True),
        StructField("city", StringType(), True),
        StructField("region", StringType(), True),
        StructField("sector_of_activity", StringType(), True),
        StructField("start_date", StringType(), True),
        StructField("end_date", StringType(), True)
    ])), True),
    StructField("skills", ArrayType(StringType()), True),
    StructField("language", ArrayType(StructType([
        StructField("name", StringType(), True),
        StructField("level", StringType(), True)
    ])), True),
    StructField("certifications", ArrayType(StructType([
        StructField("name", StringType(), True),
        StructField("etablissement_certification", StringType(), True),
        StructField("date", StringType(), True)
    ])), True)
])

# Exemple de données JSON sous forme de variable
data={
    "titre_du_poste": "Stagiaire Data Engineer (PFE)",
    "societe": "DataTech Solutions",
    "competences": [
        "Python",
        "SQL",
        "Apache Airflow",
        "Talend",
        "MongoDB",
        "Cassandra",
        "AWS",
        "Google Cloud"
    ],
    "lieu": "Lyon, France",
    "type_offre": "stage",
    "type_de_contrat": "Stage PFE",
    "durée": "6 mois",
    "email": "contact@datatechsolutions.fr",
    "telephone": "+33 1 23 45 67 89",
    "type": "hybrid",
    "langues": [
    
        "Français"
    ],
    "salaire": "1 200 ",
    "date_de_debut": "15 janvier 2025",
    "secteur_dactivite": "Technologie de l'information",
    "experience_demande": "None",
    "formation_requise": "Étudiant en informatique ou domaine connexe",
    "avantages": [
        "Assurance santé",
        "Opportunités de formation continue"
    ],
    "site_web": "www.datatechsolutions.fr"
}

# ===================================================================================
#       LOAD ENVIRONMENT VARIABLES & SET CONFIGURATIONS
# ===================================================================================
ADLS_STORAGE_ACCOUNT_NAME = "dataoffre"
ADLS_ACCOUNT_KEY = "1eNXm2As1DuaMeSt2Yjegn22fFCvIUa8nBhknayEyTgfBZb6xEEyZhnvl9OiGT7U4O7cFrygjBE/+ASt1hkNQQ=="  # Add your ADLS account key here  # Add your ADLS account key here

ADLS_CONTAINER_NAME = "offres"
ADLS_FOLDER_PATH = "offres_trav"
OUTPUT_PATH = (
    f"abfss://{ADLS_CONTAINER_NAME}@{ADLS_STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/"
    + ADLS_FOLDER_PATH
)

# Required Spark packages
PACKAGES = [
    "io.delta:delta-spark_2.12:3.0.0",
    "org.apache.hadoop:hadoop-azure:3.3.6",
    "org.apache.hadoop:hadoop-azure-datalake:3.3.6",
    "org.apache.hadoop:hadoop-common:3.3.6",
]


def create_or_get_spark(
    app_name: str, packages: List[str], cluster_manager="local[*]"
) -> SparkSession:
    jars = ",".join(packages)
    spark = (
        SparkSession.builder.appName(app_name)
        .config("spark.streaming.stopGracefullyOnShutdown", True)
        .config("spark.jars.packages", jars)
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .config(
            "fs.abfss.impl", "org.apache.hadoop.fs.azurebfs.SecureAzureBlobFileSystem"
        )
        .master(cluster_manager)
        .getOrCreate()
    )
    return spark


from pyspark.sql.functions import col, year, month, dayofmonth, hour




def create_empty_delta_table(
    spark: SparkSession,
    schema: StructType,
    path: str,
    partition_cols: Optional[List[str]] = None,
    enable_cdc: Optional[bool] = False,
):
    # Vérifier si la table Delta existe
    try:
        delta_table = DeltaTable.forPath(spark, path)
        print(f"Delta Table already exists at path: {path}")
    except Exception as e:
        print(f"Delta Table does not exist. Creating new table at: {path}")
        custom_builder = (
            DeltaTable.createIfNotExists(spark)
            .location(path)
            .addColumns(schema)
        )
        if partition_cols:
            custom_builder = custom_builder.partitionedBy(partition_cols)
        if enable_cdc:
            custom_builder = custom_builder.property(
                "delta.enableChangeDataFeed", "true"
            )

        custom_builder.execute()
        print(f"Delta table created at path: {path}")


def save_to_delta(df: DataFrame, output_path: str):
    # Sauvegarder les données traitées dans la table Delta avec la fusion de schémas
    df.write.format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .save(output_path)
    print("Data written to Delta Table")



# ===================================================================================
#                           MAIN ENTRYPOINT
# ===================================================================================

# Créer une session Spark
spark = create_or_get_spark(
    app_name="json_to_delta", packages=PACKAGES, cluster_manager="local[*]"
)

# Configurer la connexion à Azure
spark.conf.set(
    f"fs.azure.account.key.{ADLS_STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
    ADLS_ACCOUNT_KEY,
)
print("Spark Session Created")

# Lire les données JSON depuis la variable
df = spark.read.json(spark.sparkContext.parallelize([data]), schema=PERSON_SCHEMA)
print("JSON data loaded")

# Traiter les données

print("Data processed")

# Créer une table Delta vide (si elle n'existe pas encore)
create_empty_delta_table(
    spark=spark,
    schema=PERSON_SCHEMA,
    path=OUTPUT_PATH,
    partition_cols=["first_name"],
    enable_cdc=True,
)

# Sauvegarder les données traitées dans la table Delta
save_to_delta(df, OUTPUT_PATH)

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.